In [7]:
import json
import csv
from time import sleep
from kafka import KafkaConsumer, KafkaProducer

# import logging
# logging.basicConfig(level=logging.DEBUG)
const_topic = "black-fridays-3"

In [8]:
def connect_kafka_producer():
    _producer = None
    try:
        _producer = KafkaProducer(bootstrap_servers=['localhost:9092'],value_serializer=lambda m: json.dumps(m).encode('ascii'))
    except Exception as ex:
        print('Exception while connecting Kafka')
        print(str(ex))
    finally:
        return _producer

In [9]:
producer = connect_kafka_producer()

In [4]:
datasetFile = open( 'BlackFriday.csv', 'r' )  
reader = csv.DictReader( datasetFile, fieldnames = ( "user_id","product_id","gender","age","occupation","city_category","stay_in_current_city_years","marital_status","product_category_1","product_category_2","product_category_3","purchase"))   
datasetString = json.dumps( [ row for row in reader ] )  
print("JSON parsed!")  

JSON parsed!


In [5]:
datasetJson = json.loads(datasetString)[1:]
print("JSON loaded!")

JSON loaded!


In [6]:
# TEST Once
producer.send(const_topic,datasetJson[10])

In [6]:
#Sent by User ID
cacheUserId = None
cacheRows = []
for row in datasetJson:
    if cacheUserId == None:
        cacheUserId = row["user_id"]
    elif cacheUserId == row["user_id"]:
        cacheRows.append(row)
    else:
        cacheUserId = row["user_id"]
        for sub_row in cacheRows:
            producer.send(const_topic ,sub_row)
        cacheRows.clear()
        cacheRows.append(row)

In [ ]:
#Sent stream
for row in datasetJson:
    producer.send(const_topic,row)
    sleep(0.1)